In [1]:
pip install tensorflow opencv-python numpy

  Using cached tensorflow-2.19.0-cp311-cp311-win_amd64.whl.metadata (4.1 kB)
Using cached tensorflow-2.19.0-cp311-cp311-win_amd64.whl (375.9 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\bsimh\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\ext\\filters\\client_channel\\lb_policy\\grpclb\\client_load_reporting_filter.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths



In [1]:
pip install --upgrade --force-reinstall tensorflow


  Using cached tensorflow-2.19.0-cp311-cp311-win_amd64.whl.metadata (4.1 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached protobuf-5.29.4-cp310-abi3-win_amd64.whl.metadata (592 bytes)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached wrapt-1.17.2-cp311-cp311-win_amd64.whl.metadata (6.5 kB)
  Using cached grpcio-1.71.0-cp311-cp311-win_amd64.whl.metadata (4.0 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cach

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.5 requires protobuf<4,>=3.11, but you have protobuf 5.29.4 which is incompatible.


In [13]:
import zipfile
import os

zip_path = "asl_alphabet_data.zip"  # update this if the zip file is elsewhere
extract_path = "asl_alphabet_data"

# Unzip if not already extracted
if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("✅ Unzipped!")
else:
    print("✅ Already unzipped.")


✅ Already unzipped.


In [15]:
import os

print("Current working directory:", os.getcwd())
print("Files/folders here:", os.listdir())


Current working directory: c:\Users\bsimh\major
Files/folders here: ['asl_alphabet_data', 'asl_alphabet_data.zip', 'project.ipynb']


In [16]:
data_dir = "asl_alphabet_data/asl_alphabet_train"


In [17]:
import os

data_dir = "asl_alphabet_data"
print("Subfolders in asl_alphabet_data:", os.listdir(data_dir))


Subfolders in asl_alphabet_data: ['asl_alphabet_data']


In [21]:
data_dir = "asl_alphabet_data/asl_alphabet_data/asl_alphabet_data"
print("Subfolders (classes):", os.listdir(data_dir))


Subfolders (classes): ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [3]:
!pip install tqdm


In [3]:
import os
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# 🔧 Configs
data_dir = "asl_alphabet_data/asl_alphabet_data/asl_alphabet_data"
img_size = 128
batch_size = 32
initial_epochs = 10
fine_tune_epochs = 10
num_classes = 36  # A-Z + 0-9

# 🌀 Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.5, 1.5],
    shear_range=0.1,
    validation_split=0.2
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# 📦 Load Data
train_data = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_data = val_datagen.flow_from_directory(
    data_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# ⚖️ Compute Class Weights
labels = train_data.classes
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(labels), y=labels)
class_weights_dict = dict(enumerate(class_weights))

# 🧠 MobileNetV2 Base
base_model = MobileNetV2(input_shape=(img_size, img_size, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # freeze base

# 🏗️ Custom Head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)
output = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

# 🧪 Compile Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 🚀 Train Initial Model
history = model.fit(
    train_data,
    epochs=initial_epochs,
    validation_data=val_data,
    class_weight=class_weights_dict
)

# 🔓 Fine-Tune Deeper Layers
base_model.trainable = True
for layer in base_model.layers[:100]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# 🚀 Fine-Tune Model
history_fine = model.fit(
    train_data,
    epochs=fine_tune_epochs,
    validation_data=val_data,
    class_weight=class_weights_dict
)

# 💾 Save Model
model.save("asl_model_mobilenetv2_finetuned.h5")
print("✅ Saved as 'asl_model_mobilenetv2_finetuned.h5'")


Found 2012 images belonging to 36 classes.
Found 503 images belonging to 36 classes.


C:\Users\bsimh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 33s 368ms/step - accuracy: 0.1904 - loss: 3.1622 - val_accuracy: 0.7316 - val_loss: 1.0910
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 20s 315ms/step - accuracy: 0.5546 - loss: 1.3737 - val_accuracy: 0.7793 - val_loss: 0.7415
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 19s 306ms/step - accuracy: 0.6853 - loss: 0.9702 - val_accuracy: 0.8250 - val_loss: 0.5432
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 20s 319ms/step - accuracy: 0.7404 - loss: 0.7735 - val_accuracy: 0.8072 - val_loss: 0.5243
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 20s 313ms/step - accuracy: 0.7645 - loss: 0.6673 - val_accuracy: 0.8231 - val_loss: 0.4845
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 20s 313ms/step - accuracy: 0.8050 - loss: 0.5871 - val_accuracy: 0.8390 - val_loss: 0.4482
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 20s 312ms/step - accuracy: 0.8271 - loss: 0.5176 - val_accuracy: 0.8529 - val_loss: 0.3934
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 20s 310ms/step - accuracy: 0.8337 - loss: 0.4597 - val_accu

✅ Saved as 'asl_model_mobilenetv2_finetuned.h5'


In [9]:
import numpy as np
from collections import Counter

print("Samples per class:")
print(Counter(y_encoded))


Samples per class:
Counter({np.int64(5): 70, np.int64(9): 70, np.int64(4): 69, np.int64(3): 68, np.int64(15): 68, np.int64(21): 68, np.int64(20): 67, np.int64(13): 66, np.int64(7): 65, np.int64(11): 65, np.int64(30): 63, np.int64(8): 61, np.int64(1): 60, np.int64(16): 54, np.int64(35): 54, np.int64(18): 53, np.int64(27): 52, np.int64(31): 52, np.int64(6): 51, np.int64(2): 49, np.int64(32): 49, np.int64(25): 48, np.int64(33): 47, np.int64(17): 46, np.int64(34): 35, np.int64(12): 33, np.int64(0): 29, np.int64(14): 23, np.int64(10): 15, np.int64(24): 15, np.int64(19): 13, np.int64(26): 13, np.int64(28): 12, np.int64(23): 11, np.int64(22): 8, np.int64(29): 1})


ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [1]:
pip install mediapipe opencv-python


  Using cached protobuf-3.20.3-py2.py3-none-any.whl.metadata (720 bytes)
Using cached protobuf-3.20.3-py2.py3-none-any.whl (162 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install mediapipe opencv-python


Note: you may need to restart the kernel to use updated packages.


In [11]:
import matplotlib.pyplot as plt

# Plot accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


NameError: name 'history' is not defined

In [2]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model
import os

# Load trained model
model = load_model("asl_model_mobilenetv2_finetuned.h5")

# Class labels
class_labels = sorted(os.listdir("asl_alphabet_data/asl_alphabet_data/asl_alphabet_data"))
label_map = dict(enumerate(class_labels))

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)

# Start webcam
cap = cv2.VideoCapture(0)
img_size = 128

print("🎥 Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Get bounding box of the hand
            h, w, _ = frame.shape
            x_coords = [lm.x for lm in hand_landmarks.landmark]
            y_coords = [lm.y for lm in hand_landmarks.landmark]
            x1, y1 = int(min(x_coords) * w), int(min(y_coords) * h)
            x2, y2 = int(max(x_coords) * w), int(max(y_coords) * h)

            # Add some padding
            padding = 20
            x1 = max(0, x1 - padding)
            y1 = max(0, y1 - padding)
            x2 = min(w, x2 + padding)
            y2 = min(h, y2 + padding)

            # Extract and preprocess ROI
            roi = frame[y1:y2, x1:x2]
            if roi.size == 0:
                continue
            roi_resized = cv2.resize(roi, (img_size, img_size))
            roi_normalized = roi_resized.astype("float32") / 255.0
            roi_input = np.expand_dims(roi_normalized, axis=0)

            # Predict
            prediction = model.predict(roi_input)
            predicted_class = np.argmax(prediction[0])
            predicted_label = label_map[predicted_class]

            # Draw bounding box and prediction
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"Predicted: {predicted_label}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    cv2.imshow("ASL Prediction", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


🎥 Press 'q' to quit.


In [13]:
import os

data_dir = "asl_alphabet_data/asl_alphabet_data/asl_alphabet_data"
for class_name in sorted(os.listdir(data_dir)):
    class_path = os.path.join(data_dir, class_name)
    if os.path.isdir(class_path):
        print(f"{class_name}: {len(os.listdir(class_path))} images")


0: 70 images
1: 70 images
2: 70 images
3: 70 images
4: 70 images
5: 70 images
6: 70 images
7: 70 images
8: 70 images
9: 70 images
a: 70 images
b: 70 images
c: 70 images
d: 70 images
e: 70 images
f: 70 images
g: 70 images
h: 70 images
i: 70 images
j: 70 images
k: 70 images
l: 70 images
m: 70 images
n: 70 images
o: 70 images
p: 70 images
q: 70 images
r: 70 images
s: 70 images
t: 65 images
u: 70 images
v: 70 images
w: 70 images
x: 70 images
y: 70 images
z: 70 images
